In [3]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import LidarProfileFunctions as lp
import ExternalDataFunctions as ex
import GVHSRLlib as gv

import glob
import datetime

import cartopy.crs as ccrs

%matplotlib inline

/Users/mhayman/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/mhayman/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [10]:
gv_data_path = '/Users/mhayman/Documents/HSRL/GVHSRL/SOCRATES/SOCRATESrf09/'
ncep_data_path = '/Users/mhayman/Documents/Python/lidar/external_data/'

In [11]:
gv_data_files = sorted(glob.glob(gv_data_path+'SOCRATES*GVHSRL*.nc'))
hsrl_ds = xr.open_dataset(gv_data_files[0], engine='netcdf4')

In [12]:
lidar_pointing = xr.DataArray(data=hsrl_ds['lidar_pointing'].data, dims=['coordinate','time'], 
                    coords={'time': hsrl_ds['time'].data, 'coordinate':[0,1,2]},
                    name='lidar_pointing_direction',
                    attrs={'description': 'unit vector indicating the lidar\'s pointing direction'})
GGALT = xr.DataArray(data=hsrl_ds['GGALT'].data,dims=['time'],coords={'time':hsrl_ds['time'].data},
                    name='aircraft_altitude',
                    attrs={'units':'meters'})

lidar_range = hsrl_ds['range']
pixel_position_x = lidar_range*lidar_pointing.sel(coordinate=0)
pixel_position_y = lidar_range*lidar_pointing.sel(coordinate=1)
pixel_position_z = GGALT-lidar_range*lidar_pointing.sel(coordinate=2)

In [13]:
ncep_file = glob.glob(ncep_data_path+'*air*2018.nc')
ncep_ds_temp = xr.open_dataset(ncep_file[0], engine='netcdf4')
ncep_air = ncep_ds_temp['air'].sel(lat=slice(-30,-80),lon=slice(120,170),
                             time=slice(datetime.datetime(2018,1,1),datetime.datetime(2018,3,1)))

In [15]:
ncep_file = glob.glob(ncep_data_path+'*hgt*2018.nc')
ncep_ds_pres = xr.open_dataset(ncep_file[0], engine='netcdf4')
ncep_hgt = ncep_ds_pres['hgt'].sel(lat=slice(-30,-80),lon=slice(120,170),
                             time=slice(datetime.datetime(2018,1,1),datetime.datetime(2018,3,1)))

In [ ]:
#air_interp = ncep_air.interp(time=hsrl_ds['time'].data, lat=hsrl_ds['GGLON'].data, lon=hsrl_ds['GGLON'].data)
def interp_xarray(da,**kwargs):
    # accepts a data array from xarray is the interpolation data set
    # interpolates the field in da onto the grid provided in kwargs
    
    # find the bounding corners for the requested interpolation points
    plist = [[]] # list of hypercube corners
    iplist = [[]] # list of indices hypercube corners
    element_name = []
    for k in kwargs.keys():
        element_name += [k]
        # evaluate the differences between the axes points
        eval_matrix = da[k].flatten()[np.newaxis,:]-kwargs[k].flatten()[:,np.newaxis]
        
        # find the upper bound - how to handle all nan rows? (edge cases)
        pos_val = eval_matrix.copy()  # filter matrix to remove (nan) negative values
        pos_val[np.nonzero(eval_matrix < 0)] = np.nan
        pset[k]['iupper'] = np.nanargmin(pos_val,axis=0)
        pset[k]['upper'] = da[k][pset[k]['iupper']]
        #u_val+=[pset[k]['upper']]
        
        # find the lower bound
        neg_val = eval_matrix.copy()  # filter matrix to remove (nan) positive values
        neg_val[np.nonzero(eval_matrix > 0)] = np.nan
        pset[k]['ilower'] = np.nanargmax(neg_val,axis=0)
        pset[k]['lower'] = da[k][pset[k]['iupper']]
        #l_val+=[pset[k]['lower']]
        
        # create all possible permuatations of the upper/lower bounds
        # to identify the corner coordinates of the hyper cube
        p_a = []
        p_b = []
        ip_a = []
        ip_b = []
        for ai in range(len(plist)):
            p_a+=[plist[ai]+[pset[k]['upper']]]
            p_b+=[plist[ai]+[pset[k]['lower']]]
            plist = p_a+p_b
            
            ip_a+=[iplist[ai]+[pset[k]['iupper']]]
            ip_b+=[iplist[ai]+[pset[k]['ilower']]]
            iplist = ip_a+ip_b
    
    flist = []
    dlist = []
    for p in plist:
        for ai,el in enumerate(element_name):
            sel_dict[el] = p[ai]
        flist+=[da.isel(**sel_dict)]
        dlist+=np.nansum((p-x)**2)
        
    

In [37]:
import itertools
list(itertools.permutations([np.arange(4),np.arange(10,14)],2))

[(array([0, 1, 2, 3]), array([10, 11, 12, 13])),
 (array([10, 11, 12, 13]), array([0, 1, 2, 3]))]

In [55]:
l0 = [[]]
for ai in range(3):
    l1,l2=l0*2
    a1 = ai+10
    a2 = ai
    l1+=[a1]
    l2+=[a2]
    l0=l1+l2
    print(l1)
    print(l2)
    print(l0)

[10, 0]
[10, 0]
[10, 0, 10, 0]


ValueError: too many values to unpack (expected 2)

In [75]:
n0 = [[]]
xu = np.arange(3)
xl = np.arange(3)+4
for ai in range(xu.size):
    #a = list(n0)
    #b = list(n0)
    a = []
    b = []
    for el in n0:
        a+=[el+[xu[ai]]]
        b+=[el+[xl[ai]]]
        
    print(a)
    print(b)
    n0 = a+b
    print(n0)

[[0]]
[[4]]
[[0], [4]]
[[0, 1], [4, 1]]
[[0, 5], [4, 5]]
[[0, 1], [4, 1], [0, 5], [4, 5]]
[[0, 1, 2], [4, 1, 2], [0, 5, 2], [4, 5, 2]]
[[0, 1, 6], [4, 1, 6], [0, 5, 6], [4, 5, 6]]
[[0, 1, 2], [4, 1, 2], [0, 5, 2], [4, 5, 2], [0, 1, 6], [4, 1, 6], [0, 5, 6], [4, 5, 6]]


[5]